In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
! pip install geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from IPython.core.display import display, HTML
import time

C:\Users\dappi\AppData\Local\Temp\ipykernel_7052\3655213581.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
df=pd.read_csv("C:\\Users\\dappi\\Downloads\\zomato_data.csv")
df

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity
0,Yes,Yes,4.1/5,775,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari
1,Yes,No,4.1/5,787,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet,Banashankari
2,Yes,No,3.8/5,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet,Banashankari
3,No,No,3.7/5,88,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet,Banashankari
4,No,No,3.8/5,166,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...
51712,No,No,3.6 /5,27,Bar,NaN,Continental,"1,500",Pubs and bars,Whitefield
51713,No,No,NaN,0,Bar,NaN,Finger Food,600,Pubs and bars,Whitefield
51714,No,No,NaN,0,Bar,NaN,Finger Food,"2,000",Pubs and bars,Whitefield
51715,No,Yes,4.3 /5,236,Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500",Pubs and bars,Whitefield


# What is the shape of the given dataset

In [4]:
df.shape

(51717, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   online_order               51717 non-null  object
 1   book_table                 51717 non-null  object
 2   rate                       43942 non-null  object
 3   votes                      51717 non-null  int64 
 4   rest_type                  51490 non-null  object
 5   dish_liked                 23639 non-null  object
 6   cuisines                   51672 non-null  object
 7   approx_costfor_two_people  51371 non-null  object
 8   listed_intype              51717 non-null  object
 9   listed_incity              51717 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.9+ MB


# Checking null values in the dataset..

In [6]:
df.isnull().sum()

online_order                     0
book_table                       0
rate                          7775
votes                            0
rest_type                      227
dish_liked                   28078
cuisines                        45
approx_costfor_two_people      346
listed_intype                    0
listed_incity                    0
dtype: int64

In [7]:
(df.isnull().sum()/len(df))*100

online_order                  0.000000
book_table                    0.000000
rate                         15.033741
votes                         0.000000
rest_type                     0.438927
dish_liked                   54.291626
cuisines                      0.087012
approx_costfor_two_people     0.669026
listed_intype                 0.000000
listed_incity                 0.000000
dtype: float64

# Data Cleaning:

In [8]:
df['rate'].value_counts()

rate
NEW       2208
3.9/5     2098
3.8/5     2022
3.7/5     2011
3.9 /5    1874
          ... 
2.2 /5       7
2.0 /5       7
2.0/5        4
1.8 /5       3
1.8/5        2
Name: count, Length: 64, dtype: int64

In [9]:
df['rate']=df['rate'].str.replace('/5','')

In [10]:
df['rate'] = df['rate'].replace('NEW', np.nan)

In [11]:
df['rate']=df['rate'].replace('-',np.nan)

In [12]:
df['rate'].fillna(df['rate'].median(),inplace=True)

In [13]:
df['rate']=pd.to_numeric(df['rate'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   online_order               51717 non-null  object 
 1   book_table                 51717 non-null  object 
 2   rate                       51717 non-null  float64
 3   votes                      51717 non-null  int64  
 4   rest_type                  51490 non-null  object 
 5   dish_liked                 23639 non-null  object 
 6   cuisines                   51672 non-null  object 
 7   approx_costfor_two_people  51371 non-null  object 
 8   listed_intype              51717 non-null  object 
 9   listed_incity              51717 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.9+ MB


In [15]:
# Step 2: Cost Column (approx_costfor_two_people)
df['approx_costfor_two_people']=df['approx_costfor_two_people'].str.replace("," , "")

In [16]:
df['approx_costfor_two_people']=pd.to_numeric(df['approx_costfor_two_people'])

In [17]:
df['approx_costfor_two_people'].fillna(df['approx_costfor_two_people'].median(),inplace=True)

# Step 3: Categorical Columns
dish_liked → Replace NaN with "Not Available"


cuisines → Replace NaN with "Other"


rest_type → Replace NaN with "Unknown"




In [18]:
df['dish_liked']=df['dish_liked'].replace(np.nan,"Not Available")

In [19]:
df['cuisines']=df['cuisines'].replace(np.nan,"Other")

In [20]:
df['rest_type']=df['rest_type'].replace(np.nan,"Unknown")

# Step 4: Votes Column
Fill missing values in votes with median


In [21]:
df['votes'].fillna(df['votes'].median())

0        775
1        787
2        918
3         88
4        166
        ... 
51712     27
51713      0
51714      0
51715    236
51716     13
Name: votes, Length: 51717, dtype: int64

# Step 5: Binary Encoding
Convert the following binary fields:
Column
Mapping
online_order
Yes → 1, No → 0
book_table
Yes → 1, No → 0



In [22]:
df['online_order']=df['online_order'].replace(['Yes','No'],[1,0])

In [23]:
df['book_table']=df['book_table'].replace(['Yes','No'],[1,0])

# Step 6: Data Type Conversion
Ensure the following conversions:
rate → float


votes → integer


approx_costfor_two_people → integer




In [24]:
df['approx_costfor_two_people']=df['approx_costfor_two_people'].astype(int)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   online_order               51717 non-null  int64  
 1   book_table                 51717 non-null  int64  
 2   rate                       51717 non-null  float64
 3   votes                      51717 non-null  int64  
 4   rest_type                  51717 non-null  object 
 5   dish_liked                 51717 non-null  object 
 6   cuisines                   51717 non-null  object 
 7   approx_costfor_two_people  51717 non-null  int32  
 8   listed_intype              51717 non-null  object 
 9   listed_incity              51717 non-null  object 
dtypes: float64(1), int32(1), int64(3), object(5)
memory usage: 3.7+ MB


In [26]:
# 1. What is the shape of the given dataset?
df.shape

(51717, 10)

In [27]:
# 2. How many restaurants serve North Indian cuisine?
df[df['cuisines']=='North Indian']

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity
5,1,0,3.8,286,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,Buffet,Banashankari
50,0,0,3.7,21,Quick Bites,Not Available,North Indian,450,Delivery,Banashankari
72,0,0,3.7,0,Quick Bites,Not Available,North Indian,150,Delivery,Banashankari
87,0,0,3.5,31,Quick Bites,Not Available,North Indian,200,Delivery,Banashankari
94,0,0,2.6,283,Casual Dining,"Biryani, Chicken Guntur, Thali, Buttermilk, Ma...",North Indian,600,Delivery,Banashankari
...,...,...,...,...,...,...,...,...,...,...
51622,1,0,3.5,17,Quick Bites,Not Available,North Indian,250,Dine-out,Whitefield
51626,0,0,3.7,0,Quick Bites,Not Available,North Indian,300,Dine-out,Whitefield
51627,0,0,3.7,0,Quick Bites,Not Available,North Indian,300,Dine-out,Whitefield
51631,0,0,3.4,5,Quick Bites,Not Available,North Indian,250,Dine-out,Whitefield


In [28]:
n=df[df['cuisines'].str.contains('North Indian')]

In [29]:
n.shape

(21085, 10)

In [30]:
# What cuisine is most commonly offered by restaurants in Bangalore?
df['cuisines'].value_counts().nlargest()

cuisines
North Indian             2913
North Indian, Chinese    2385
South Indian             1828
Biryani                   918
Bakery, Desserts          911
Name: count, dtype: int64

In [31]:
df1=pd.read_csv(r"C:\Users\dappi\Downloads\Geographical Coordinates.csv")

In [32]:
df1

,listed_incity,Latitude,Longitude
0,Banashankari,12.939333,77.553982
1,Bannerghatta Road,12.952660,77.605048
2,Basavanagudi,12.941726,77.575502
3,Bellandur,12.925352,77.675941
4,Brigade Road,12.967358,77.606435
5,Brookefield,12.963814,77.722437
6,BTM,12.916360,77.604733
7,Church Street,12.974914,77.605247
8,Electronic City,12.848760,77.648253
9,Frazer Town,12.998683,77.615525


In [33]:
# Which locality in Bangalore has the highest average cost for dining (for two people)?
a=df.groupby('listed_incity')['approx_costfor_two_people'].mean()
a.nlargest(1)

listed_incity
Church Street    770.361248
Name: approx_costfor_two_people, dtype: float64

In [34]:
# 5. Which restaurant type has the top rating with over 1000 votes?
over_1000_votes=df[df['votes']>1000]
over_1000_votes.groupby('rest_type')['rate'].mean().nlargest()

rest_type
Bakery                  4.800000
Pub, Cafe               4.657500
Cafe, Dessert Parlor    4.578261
Dessert Parlor          4.518367
Microbrewery            4.508333
Name: rate, dtype: float64

In [35]:
#  How much does it cost at minimum to eat out in Bangalore?
df.groupby('listed_incity')['approx_costfor_two_people'].min()

listed_incity
BTM                       70
Banashankari              80
Bannerghatta Road         50
Basavanagudi             100
Bellandur                100
Brigade Road              40
Brookefield              100
Church Street             40
Electronic City           60
Frazer Town               40
HSR                      100
Indiranagar               40
JP Nagar                  70
Jayanagar                 80
Kalyan Nagar              80
Kammanahalli              80
Koramangala 4th Block     40
Koramangala 5th Block     40
Koramangala 6th Block     40
Koramangala 7th Block    100
Lavelle Road              50
MG Road                   50
Malleshwaram             100
Marathahalli             100
New BEL Road             100
Old Airport Road         100
Rajajinagar               80
Residency Road            50
Sarjapur Road            100
Whitefield               100
Name: approx_costfor_two_people, dtype: int32

In [36]:
# 7. What percentage of total online orders is received by restaurants in Banashankari?
ban = df[(df['listed_incity'] == 'Banashankari')]
percentage= (len(ban[ban['online_order']==1])/len(ban))*100
percentage

63.26767091541136

In [37]:
ban = df[(df['online_order'] == 1)]
percentage= (len(ban[ban['listed_incity']=='Banashankari'])/len(ban))*100
percentage

1.7934568387859677

In [38]:
# 8. Which locality has the most restaurants with over 500 votes and a rating below 3.0?
most_rest = df[(df['votes']>500) & (df['rate']<3)]
most_rest['listed_incity'].value_counts().nlargest(1)

listed_incity
Brookefield    8
Name: count, dtype: int64

In [39]:
# 9. Which locality in Bangalore should Zomato target for expansion based on restaurant type diversity?
df.groupby('listed_incity')['rest_type'].nunique().nlargest(3)

listed_incity
BTM              62
Brigade Road     61
Church Street    61
Name: rest_type, dtype: int64

In [40]:
# 10. What's the average cost difference between buffet and delivery restaurants?
buffet=df[df['listed_intype']=='Buffet']
delivery = df[df['listed_intype']=='delivery']
buf_avg = buffet['approx_costfor_two_people'].mean()
del_avg = delivery['approx_costfor_two_people'].mean()
buf_avg


1295.3514739229024

In [41]:
# 11 . What is the maximum number of votes received by any restaurant with online ordering?
online_order = df[df['online_order']==1]
online_order['votes'].max()

16832

In [42]:
# 12. What is the average rating of restaurants that serve both North Indian and Chinese cuisines?
serve_both=df[df['cuisines'].str.contains('North Indian') & df['cuisines'].str.contains('Chinese')]
serve_both['rate'].mean()

3.5919080217658412

In [43]:
# 13. What is the most profitable area for Zomato based on potential revenue estimation?
df['potential_revenue'] = df['approx_costfor_two_people'] * df['votes']
area_revenue = df.groupby('listed_incity')['potential_revenue'].sum()
most_profitable_area = area_revenue.sort_values(ascending=False).head(1)
most_profitable_area

listed_incity
Koramangala 7th Block    1006195610
Name: potential_revenue, dtype: int64

In [44]:
# 14. If Zomato wants to reduce customer complaints, which restaurant type should they focus on?
low_rated = df[df['rate'] < 3.5]
complaint_risk = low_rated['rest_type'].value_counts().nlargest(3)
complaint_risk

rest_type
Quick Bites      5429
Casual Dining    2128
Delivery          641
Name: count, dtype: int64

In [45]:
# 15. In which area should Zomato invest by considering high rating (rate > 4.2), high number of votes (> 500) and including online orders?
invest=df[(df['rate']>4.2) & (df['votes']>500) & (df['online_order']==1)]
invest['listed_incity'].value_counts().nlargest(5)

listed_incity
Koramangala 7th Block    97
MG Road                  97
Church Street            95
Koramangala 4th Block    95
Brigade Road             94
Name: count, dtype: int64

In [46]:
merged_df=pd.merge(df, df1, on='listed_incity', how='left')

In [47]:
merged_df

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_costfor_two_people,listed_intype,listed_incity,potential_revenue,Latitude,Longitude
0,1,1,4.1,775,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari,620000,12.939333,77.553982
1,1,0,4.1,787,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet,Banashankari,629600,12.939333,77.553982
2,1,0,3.8,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet,Banashankari,734400,12.939333,77.553982
3,0,0,3.7,88,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet,Banashankari,26400,12.939333,77.553982
4,0,0,3.8,166,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet,Banashankari,99600,12.939333,77.553982
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,0,0,3.6,27,Bar,Not Available,Continental,1500,Pubs and bars,Whitefield,40500,NaN,NaN
51713,0,0,3.7,0,Bar,Not Available,Finger Food,600,Pubs and bars,Whitefield,0,NaN,NaN
51714,0,0,3.7,0,Bar,Not Available,Finger Food,2000,Pubs and bars,Whitefield,0,NaN,NaN
51715,0,1,4.3,236,Bar,"Cocktails, Pizza, Buttermilk",Finger Food,2500,Pubs and bars,Whitefield,590000,NaN,NaN


In [54]:
!pip install folium
!pip install IPython

   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/113.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/113.4 kB ? eta -:--:--
   ---------- ---------------------------- 30.7/113.4 kB 435.7 kB/s eta 0:00:01
   ------------------------ -------------- 71.7/113.4 kB 653.6 kB/s eta 0:00:01
   -------------------------------------  112.6/113.4 kB 930.9 kB/s eta 0:00:01
   -------------------------------------- 113.4/113.4 kB 599.7 kB/s eta 0:00:00


In [57]:
from folium.plugins import MarkerCluster
import folium 
from IPython.display import IFrame

In [52]:
italian_df = df[df['cuisines'].str.contains('Italian')]


In [64]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   online_order               51717 non-null  int64  
 1   book_table                 51717 non-null  int64  
 2   rate                       51717 non-null  float64
 3   votes                      51717 non-null  int64  
 4   rest_type                  51717 non-null  object 
 5   dish_liked                 51717 non-null  object 
 6   cuisines                   51717 non-null  object 
 7   approx_costfor_two_people  51717 non-null  int32  
 8   listed_intype              51717 non-null  object 
 9   listed_incity              51717 non-null  object 
 10  potential_revenue          51717 non-null  int64  
 11  Latitude                   46137 non-null  float64
 12  Longitude                  46137 non-null  float64
dtypes: float64(3), int32(1), int64(4), object(5)
m

In [67]:
df_clean = merged_df.dropna(subset=['Latitude', 'Longitude'],inplace=True)

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   online_order               51717 non-null  int64  
 1   book_table                 51717 non-null  int64  
 2   rate                       51717 non-null  float64
 3   votes                      51717 non-null  int64  
 4   rest_type                  51717 non-null  object 
 5   dish_liked                 51717 non-null  object 
 6   cuisines                   51717 non-null  object 
 7   approx_costfor_two_people  51717 non-null  int32  
 8   listed_intype              51717 non-null  object 
 9   listed_incity              51717 non-null  object 
 10  potential_revenue          51717 non-null  int64  
dtypes: float64(1), int32(1), int64(4), object(5)
memory usage: 4.1+ MB
